<a href="https://colab.research.google.com/github/pradeep004/Udemy-Python-Programming/blob/main/Assignement_ML_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



#Machine Learning Assignment



In [ ]:
@Auther      : Pradeep Angadageri 
@Date_edited : 05/05/2021
@Roll_Number : 

In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import hstack
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer
import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle
from tqdm import tqdm
import os


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Loading of the "Russia Investigation dataset"
data = pd.read_csv("/content/drive/My Drive/Masters/russia-investigation.csv")

In [ ]:
#Information about the columns and deciding which column will be helpful to predict the output.

#Note - Columns marked as "Important Feature" will be considered for model building.

#Columns with its description as per my understanding 
#1. investigation       : This tells what we are investigating, this can be considered as a id (Important Feature) 
#2. investigation-start : The date when investigation started (Not Important Feature)
#3. investigation-end   : The date when investigation ended (Not Important Feature)
#4. investigation-days  : difference between start date and end date of the investigation (Important Feature)
#5. name                : Name of the person who is charged in the investigation (Not Important Feature) 
#6. indictment-days     : Length of the days between start date and charged date (Important Feature)
#7. type                : Result of charge (Important Feature)
#8. cp-date             : Date the person pled gulty or convicted if applicable (Not Importnat Feature)
#9  cp-days             : Lenght of start date of the investigation to person pled gulty or convicted (Importnat Feature)
#10 Overturned          : Conviction overturned or not (Importnat Feature)
#11 American            : Charged person is america or not (Important Feature)
#12 President           : WHo was the presedent during the investigation and person charged (Important Feature)
#13 Pardoned            : Whether the person pardoned or not (Important Feature)

##Task

######Problem Statement : Using Regression and Classification Alogirthm , Predict whether the person is pardoned or not based on the input columns or data 


In [ ]:
#Steps that we will follow to complete this assignemnt 
#1)Remove Useless features from the data set 
#2)featurization of the columns 
#3)Applying Machine learning Algorithm 
#4)Measure the Models accuracy 

In [ ]:
#Removing Useless Features from the dataset and taking only userfull features for preduction 

#Below are the reasons why features are important 
#1. investigation-start : Since the difference between start date and end date is given , this feature is not important
#2. investigation-end   : Since the difference between start date and end date is given , this feature is not important
#3. name                : With name we cannot make the model better, hence removing this feature.
#4. cp-date             : Difference between start date and cp-date is given hence this feature can be removed.

#Code to remove above four columns from the dataset
data = data.drop(columns=["investigation-start","investigation-end","name","cp-days","cp-date"])

In [ ]:
data

,investigation,investigation-days,indictment-days,type,overturned,pardoned,american,president
0,watergate,1492,-246.0,conviction,False,False,True,Richard Nixon
1,watergate,1492,-246.0,conviction,False,False,True,Richard Nixon
2,watergate,1492,292.0,conviction,True,False,True,Richard Nixon
3,watergate,1492,-246.0,guilty-plea,False,True,True,Richard Nixon
4,watergate,1492,292.0,conviction,True,False,True,Richard Nixon
...,...,...,...,...,...,...,...,...
189,russia,659,422.0,indictment,False,False,False,Donald Trump
190,russia,659,422.0,indictment,False,False,False,Donald Trump
191,russia,659,422.0,indictment,False,False,False,Donald Trump
192,russia,659,561.0,guilty-plea,False,False,True,Donald Trump


In [ ]:
data.columns

Index(['investigation', 'investigation-days', 'indictment-days ', 'type',
       'overturned', 'pardoned', 'american', 'president'],
      dtype='object')

In [ ]:
#Preprocessing 
data = data.rename(columns={'indictment-days ': 'indictment-days'})
#Checked nan in all the columns and found 13 nan values in indictment-days , below code removes those rows 
data.dropna(subset = ["indictment-days"], inplace=True)

In [ ]:
data.columns

Index(['investigation', 'investigation-days', 'indictment-days', 'type',
       'overturned', 'pardoned', 'american', 'president'],
      dtype='object')

In [ ]:
#data['indictment-days'].isnull().values.any() # code to check nan values 
#data['indictment-days'].isnull().sum() # code to check count of nan values 

In [ ]:
#Seperate Input and output label and divide them into train and test data 
X = data[["investigation","investigation-days","indictment-days","type","overturned","american","president"]]
Y = data["pardoned"]

#Split data into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33) # this is random splitting

#Printing shapes after splitting the data 
print("*******************Printing shapes after splitting the dataset, for better understaing*******************")
print("Before splitting : ",data.shape)
print("After splitting :")
print("X_train : ",X_train.shape)
print("y_train : ",y_train.shape)
print("X_test  : ",X_test.shape)
print("y_test  : ",y_test.shape)

*******************Printing shapes after splitting the dataset, for better understaing*******************
Before splitting :  (181, 8)
After splitting :
X_train :  (121, 7)
y_train :  (121,)
X_test  :  (60, 7)
y_test  :  (60,)


In [ ]:
 # one-hot encoding of Investigation feature.
inve_vectorizer = CountVectorizer()
train_inve_feature_onehotCoding = inve_vectorizer.fit_transform(X_train["investigation"])
test_inve_feature_onehotCoding = inve_vectorizer.transform(X_test["investigation"])


#Converted investigation word feature to one hot encoded feature
print(train_inve_feature_onehotCoding.shape)
print(test_inve_feature_onehotCoding.shape)

(121, 9)
(60, 9)


In [ ]:
#Normal distribution of investigation-days feature
from sklearn import preprocessing
import pandas as pd

x = X_train["investigation-days"]
x = x.to_numpy()
x = x.reshape(-1,1)

y = X_test["investigation-days"]
y = y.to_numpy()
y = y.reshape(-1,1)

min_max_scaler = preprocessing.MinMaxScaler()

min_max_scaler.fit(x)
x_scaled = min_max_scaler.transform(x)
y_scaled = min_max_scaler.transform(y)


X_train["investigation-days"] = x_scaled
X_test["investigation-days"] = y_scaled

In [ ]:
#Normal distribution of indictment-days	 feature
from sklearn import preprocessing
import pandas as pd

x = X_train["indictment-days"]
x = x.to_numpy()
x = x.reshape(-1,1)

y = X_test["indictment-days"]
y = y.to_numpy()
y = y.reshape(-1,1)

min_max_scaler = preprocessing.MinMaxScaler()

min_max_scaler.fit(x)
x_scaled = min_max_scaler.transform(x)
y_scaled = min_max_scaler.transform(y)


X_train["indictment-days"] = x_scaled
X_test["indictment-days"] = y_scaled

In [ ]:
 # one-hot encoding of type feature.
type_vectorizer = CountVectorizer()
train_type_feature_onehotCoding = type_vectorizer.fit_transform(X_train["type"])
test_type_feature_onehotCoding = type_vectorizer.transform(X_test["type"])


#Converted investigation word feature to one hot encoded feature
print(train_type_feature_onehotCoding.shape)
print(test_type_feature_onehotCoding.shape)

(121, 4)
(60, 4)


In [ ]:
#Mapping True/False to 0/1 for overtuned feature
X_train["overturned"] = X_train["overturned"].astype(int)
X_test["overturned"] = X_test["overturned"].astype(int)


#Mapping True/False to 0/1 for pardoned feature
y_train = y_train.astype(int)
y_test = y_test.astype(int)

#Mapping True/False to 0/1 for american feature
X_train["american"] = X_train["american"].astype(int)
X_test["american"] = X_test["american"].astype(int)

In [ ]:
 # one-hot encoding of type feature.
pre_vectorizer = CountVectorizer()
train_pre_feature_onehotCoding = pre_vectorizer.fit_transform(X_train["president"])
test_pre_feature_onehotCoding = pre_vectorizer.transform(X_test["president"])


#Converted investigation word feature to one hot encoded feature
print(train_pre_feature_onehotCoding.shape)
print(test_pre_feature_onehotCoding.shape)

(121, 10)
(60, 10)


In [ ]:
train_onehotCoding = hstack((train_inve_feature_onehotCoding,train_type_feature_onehotCoding,train_pre_feature_onehotCoding))
test_onehotCoding = hstack((test_inve_feature_onehotCoding,test_type_feature_onehotCoding,test_pre_feature_onehotCoding))

In [ ]:
#training data

#Working with investigation_days column
investigation_days = np.array(X_train["investigation-days"].values.tolist())
investigation_days = investigation_days.reshape(investigation_days.shape[0],1)

#Working with indictment_days column
indictment_days = np.array(X_train["indictment-days"].values.tolist())
indictment_days = indictment_days.reshape(indictment_days.shape[0],1)

#Working with overturned column
overturned = np.array(X_train["overturned"].values.tolist())
overturned = overturned.reshape(overturned.shape[0],1)

#Working with american column
american = np.array(X_train["american"].values.tolist())
american = american.reshape(american.shape[0],1)

In [ ]:
train_onehotCoding = hstack((train_onehotCoding,investigation_days,indictment_days,overturned,american)).tocsr()

In [ ]:
#testing data

#Working with investigation_days column
investigation_days = np.array(X_test["investigation-days"].values.tolist())
investigation_days = investigation_days.reshape(investigation_days.shape[0],1)

#Working with indictment_days column
indictment_days = np.array(X_test["indictment-days"].values.tolist())
indictment_days = indictment_days.reshape(indictment_days.shape[0],1)

#Working with overturned column
overturned = np.array(X_test["overturned"].values.tolist())
overturned = overturned.reshape(overturned.shape[0],1)

#Working with american column
american = np.array(X_test["american"].values.tolist())
american = american.reshape(american.shape[0],1)

In [ ]:
test_onehotCoding = hstack((test_onehotCoding,investigation_days,indictment_days,overturned,american)).tocsr()

###ML Algorithm : KNN 

In [ ]:
from sklearn.metrics.classification import accuracy_score, log_loss
def predict_and_plot_confusion_matrix(train_x, train_y,test_x, test_y,clf):
 clf.fit(train_x, train_y)
 sig_clf = CalibratedClassifierCV(clf, method="sigmoid")
 sig_clf.fit(train_x, train_y)
 pred_y = sig_clf.predict(test_x)
 # for calculating log_loss we willl provide the array of probabilities belongs to each class
 print("Log loss :",log_loss(test_y, sig_clf.predict_proba(test_x)))
 # calculating the number of data points that are misclassified
 print("Number of mis-classified points :", np.count_nonzero((pred_y- test_y))/test_y.shape[0])

In [ ]:
#Applying classification algorithm that is KNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.calibration import CalibratedClassifierCV
clf = KNeighborsClassifier(n_neighbors=10)
predict_and_plot_confusion_matrix(train_onehotCoding, y_train, test_onehotCoding, y_test, clf)

Log loss : 0.4046278112685952
Number of mis-classified points : 0.16666666666666666


###ML Algorithm : Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(train_onehotCoding, y_train)
print("Linear Regression Score : ",reg.score(train_onehotCoding, y_train))
print("\n")

print("Regression Coeffient : \n",reg.coef_)

print("\n")
print("Final output of Linear Regression\n",reg.predict(test_onehotCoding))

Linear Regression Score :  0.3633201199734575


Regression Coeffient : 
 [-5.64923936e+02 -9.74834828e+01  3.06073379e+02 -9.74834828e+01
 -8.63814791e+02  7.93221041e+02  3.55465135e+02  1.25695447e+02
 -5.42327937e+01  5.34657611e-03  1.00071549e-01 -1.05418152e-01
  1.00071549e-01 -3.13083350e+02 -7.05937493e+01 -3.13083350e+02
  3.55465135e+02 -7.05937493e+01  1.25695447e+02 -9.74834828e+01
  1.25695447e+02 -9.74834828e+01  3.55465135e+02  2.25772625e+03
  9.93862050e-02 -5.18132935e-02 -2.51254629e-01]


Final output of Linear Regression
 [-1.49460853e-01 -7.82036665e-02  3.37760676e-01  1.85488145e-01
 -1.61949810e-01  9.02281756e-02 -3.49692691e-03 -5.14864047e-02
  4.93146301e-01  1.00422920e-01  2.00956034e-01 -1.99349844e-01
  3.48758947e-01  1.16592855e-01  1.53354200e-01 -1.02998376e-01
 -1.67351264e+03  3.88547437e-03  1.36229038e-01  1.96925050e-01
  3.15713913e-01  4.65273970e-01  1.51546265e-01  3.88547437e-03
 -8.34457528e+02  1.39694247e-01 -5.14864047e-02  3.88547437